## **IMPORT DATASET**

In [ ]:
# Importing Dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Train Data
df =  pd.read_excel('your excel dataset')
print("Data shape:",df.shape)
NegatifDf = df[df["label"]=="negative"].drop("label",axis=1)
NegatifDf["label"] = 0

PositifDf = df[df["label"]=="positive"].drop("label",axis=1)
PositifDf["label"] = 1

df = pd.concat([NegatifDf,PositifDf],ignore_index=True)
print("Final total dataset:",df.shape)


## **PRE-PROCESSING DATA**

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [ ]:
stop=set(stopwords.words("indonesian"))
stop_bow = set(stopwords.words("indonesian"))
stop_bow.discard("tidak")
factory = StemmerFactory()
stemmer = factory.create_stemmer()

text=df["tweet"]
cleaned_text_bow=[]
cleaned_text=[]
for line in text:
    tags = re.compile("^@[a-zA-Z_]*")
    line = re.sub(tags," ",line)
    hashtags = re.compile("#|\*")
    extraCharacters = re.compile("[^a-zA-Z]")
    line = re.sub(extraCharacters," ",line)

    filtered_words=[]
    filtered_words_bow=""
    for word in line.split():
        word=word.lower()
        if(word not in stop):
            word = stemmer.stem(word)
            filtered_words.append(word)
        if(word not in stop_bow):
            word = stemmer.stem(word)
            filtered_words_bow+=" "+word
    cleaned_text.append(filtered_words)
    cleaned_text_bow.append(filtered_words_bow)
            
data_bow = pd.DataFrame(data=cleaned_text_bow,columns=["tweet"])
data_bow["label"] = df["label"]
df["tweet"]=cleaned_text
print(df.head(10))
print()
print(data_bow.head(10))
print()

## **WORD2VEC FEATURE EXTRACTION**

### Word2Vec Process

In [ ]:
import gensim
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

In [ ]:
#Train on text data
list_of_sent = df["tweet"]
w2v_model=gensim.models.Word2Vec(list_of_sent, window=5, vector_size=100, workers=4, sg=1, negative=17) #size = ukuran vector yg diinginkan, default cbow
out_weights = w2v_model.syn1neg
#out_weights
# corpus/vocab ambil dr kata yg ada di data trainw2v_words=list(w2v_model.wv.index_to_key) 
w2v_words=list(w2v_model.wv.index_to_key) 

In [ ]:
# Vectorize train text data
listof_sent_vec=[]
for sent in tqdm(list_of_sent): 
    sent_vec = np.zeros(100) 
    cnt_words =0; 
    for word in sent: 
        if word in w2v_words:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    listof_sent_vec.append(sent_vec)
    
Label = df["label"]
list_col=tuple(range(100))
Scaler = StandardScaler()
data_vec = Scaler.fit_transform(listof_sent_vec)
W2v_data=pd.DataFrame(data=data_vec, columns=list_col)
W2v_data["label"] = Label
#print(W2v_data.head(10))
#print(W2v_data.shape) 

### Mapping X dan y Variables

In [ ]:
X = W2v_data.drop("label",axis=1).to_numpy()

In [ ]:
y = W2v_data["label"].to_numpy()
y[0:5]

In [ ]:
#Jalankan ini untuk dapatkan indeks awal pada hasil splitting data, tapi hapus untuk dapatkan idnex data hasil split baru cocokkan
y = pd.Series(Label)
y[0:5]

## **SVM CLASSIFICATION**

In [ ]:
from sklearn import model_selection, svm
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from scipy.sparse import csr_matrix as matrixTransform

### Worked Code for reducted and not-reducted Data - Kfold CV

In [ ]:
#FIXX CODE
# # Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# Importing the dataset
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=0)
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
# Fitting Kernel SVM to the Training set
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, y_train)
# Predicting the Test set results
y_pred = classifier.predict(X_test)
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
#get train proportion
import numpy
unique, counts = numpy.unique(y_test, return_counts=True)
dict(zip(unique, counts))

In [ ]:
#visualisasikan confusion matrix
import matplotlib.pyplot as plt
import numpy
from sklearn import metrics

cm = confusion_matrix(y_test, y_pred)
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = [0, 1])
cm_display.plot()
plt.show()

In [ ]:
FN = cm[1][0]
FP = cm [0][1]
TP = cm[1][1]
TN = cm[0][0]
(TP, TN, FP, FN)

In [ ]:
#get index data of confusion matrix element
FP = cm [0][1]
FP_rows = []

for i in range(len(y_pred)):
    if y_pred[i] == 1 and y_test[i] == 0:
        FP_rows.append(i)
FP_rows

In [ ]:
#untuk dapatkan data belum bersihnya
data = pd.read_csv('for_preprocessingg.csv', encoding='latin-1')

In [ ]:
#label dari dataset
display(df["label"].iloc[27])

In [ ]:
#data belum preprocessing
display(data["tweet"].iloc[27])

In [ ]:
#data bersih
display(df["tweet"].iloc[27])

## Preparing Data for LDA Modelling